<a href="https://colab.research.google.com/github/elaineye117/flight_crash_analysis/blob/master/SI_649_Common_Causes_of_Failures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SI 649 Fun With Python

In [15]:
#sources: https://pythonspot.com/tokenizing-words-and-sentences-with-nltk/,
#python.org/documenation
#https://www.nltk.org/book/
#stackoverflow here and there 

import nltk
import pandas as pd
import numpy as np

file2 = 'https://raw.githubusercontent.com/lhamini/VisQ_Aairplane/master/Airplane_Crashes_and_Fatalities_Since_1908_20190820105639.csv'
safety = pd.read_csv(file2, sep=',', encoding="latin-1")
safety.head() 

#Some simple facts about the dataset #2 
#Total occurances per filed 
airplane_info2 = pd.read_csv(file2)
airplane_info2.head()

data_subset = airplane_info2[['Operator','Summary']]
data_subset.head()

#Current data set size
print("Original data set size: ",len(data_subset))
#Drop nan's
no_nans= data_subset.dropna()
print("Size after drop nans: ",len(no_nans))

operator = no_nans['Operator']
summary = no_nans['Summary']

#Now we need to loop through each summary clean it from stop-words & punctuation
#Stop words, are just words without any inheriten meaning ( and, so, ...)
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import string 

#List of stop words & punctuation symbols 
nltk.download('punkt')
nltk.download('stopwords')
stop = stopwords.words('english')
more_stop_words = ['the', '.',"'s"]
punct = string.punctuation

#summaries without stop words & punctuation & numbers 
#determine most important [words, frequency, operator] for each summary

clean_summaries= []

for x in range(len(summary)):
    example_sent =summary.iloc[x]
    word_tokens = word_tokenize(example_sent) 
    word_tokens2=[x.lower() for x in word_tokens]
    filtered_sentence = [w for w in word_tokens2 if not w in stop and w not in punct and w not in more_stop_words and not w.isdigit() ]
    #remove duplicates before appending
    filtered_sentence_2 = list(set(filtered_sentence))
    
    token= word_tokenize(operator.iloc[x])
    #determine if military, private, or commercial
    tha_type=""
    if(token[0].lower()== 'military'):
        tha_type = 'military'
        
    elif(token[0].lower()== 'private'):
        tha_type = 'private'
        
    else:
        tha_type = 'commercial'
    
    for y in filtered_sentence_2:
        temp = (y, 1, tha_type)
        clean_summaries.append(temp)

#Stemming
#crashing become crash
from nltk.stem import PorterStemmer
clean_summary_2 = []
ps = PorterStemmer()

for word in clean_summaries:
    w= ps.stem(word[0])
    if(w not in stop and w not in punct and w not in more_stop_words and not w.isdigit()):
        temp = (w, word[1], word[2])
        clean_summary_2.append(temp)
  

Original data set size:  4967
Size after drop nans:  4894
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
#Top 65 words
top_fifty=[      
'weather','fog','rain','ice','thunderstorm','wind','turbul','cloud','snow','storm','lightn','snowstorm','windshear', 'rainstorm','sandstorm',
 'failur','accid','error','problem','malfunct','crack','leak', 
  "engin",'rotor','rudder', 'wing','brake','skid','tire','fuselag',
  'shot','explod','explos','deton','missil','bomb',
    "mountain",'mountainsid','mount','cliff','ravin',
     'struck','hit','impact','collis',
      'cargo','weight', 'overload',
      'ground','takeoff', 'take-Off',
       'hijack','sabotag' ,'attack',  
       'fire','flame','burn', 
'lost', 
'tree',
'fuel',
'visibl',
'fatigu',
'autopilot',
'volcano',
'overspeed',       
]

top_causes = []
#Append each of the top 70 words in a new list, for every time it appears in the data set
for x in clean_summary_2:
  if(x[0] in top_fifty):
    top_causes.append(x)
   
#Sort so that it appears uniform colored in the graaph
s= sorted(top_causes, key=lambda i: ( i[0], i[2] ), reverse=False)

#Create dataset & Upload on GitHub
df2 = pd.DataFrame(s)
df2.to_csv('top_65.csv', index=False)

#Wilkinson Chart
alt.Chart(df2,title = "TOP 70 CAUSES").mark_circle().transform_window(
    id='rank()',
    groupby=['0']
).encode(
    alt.Y('0:O', sort=alt.EncodingSortField(field="id",order='descending')),
    alt.X('id:O'),
    color=alt.Color('2:O', legend=alt.Legend(title="Legend",orient="left"),scale=alt.Scale(scheme='set1') )
     
).configure_axis(
    grid=False
)

alt.Chart(...)

In [0]:
# #Check out the same chart done in Tableau
# #Log Scale
https://public.tableau.com/views/FrequentCrashCauses-LogScale/Sheet1?:display_count=y&:origin=viz_share_link


# #Normal Scale
https://public.tableau.com/views/FrequentCrashCauses-NormalScale/Sheet12?:display_count=y&:origin=viz_share_link
